In [ ]:
%load_ext autoreload
%autoreload 2
from Robot.sawyer import Sawyer
import rospy
import numpy as np
from franka_interface.robot_enable import RobotEnable

rospy.init_node('notebook_node')
robot = Sawyer()
robot_enable  = RobotEnable()

##### Error Recovery

In [ ]:
!rostopic pub -1 /franka_ros_interface/franka_control/error_recovery/goal franka_msgs/ErrorRecoveryActionGoal "{}"

In [ ]:
robot_enable.enable() 

#### Test Joint Position Control

In [ ]:
from Common import config
from Robot import kdl_utils

bottleneck_pose_vector_vertical = np.load('../Data/' + str(config.TASK_NAME) + '/Single_Demo/Raw/bottleneck_pose_vector_vertical.npy')
bottleneck_pose_vector = np.load('../Data/' + str(config.TASK_NAME) + '/Single_Demo/Raw/bottleneck_pose_vector.npy')

bottleneck_pose_vertical = kdl_utils.create_pose_from_vector(bottleneck_pose_vector_vertical)
bottleneck_pose = kdl_utils.create_pose_from_vector(bottleneck_pose_vector)

#print('{},{}'.format(bottleneck_pose_vertical.M.GetRPY(),bottleneck_pose.M.GetRPY()))
target_pose_3dof = kdl_utils.create_pose_3dof_from_pose(bottleneck_pose_vertical)
bottleneck_pose = kdl_utils.create_vertical_pose_from_x_y_z_theta(target_pose_3dof[0], target_pose_3dof[1], bottleneck_pose_vertical.p[2], target_pose_3dof[2])
print('{},{}'.format(bottleneck_pose_vertical.M.GetRPY(),bottleneck_pose.M.GetRPY()))
#robot.move_to_pose(bottleneck_pose)

In [ ]:
robot.control_manager.set_motion_controller(robot.control_manager.default_controller)

In [ ]:
#robot.robot.set_command_timeout(10.0)
from Common import config
robot.move_to_joint_angles(config.ROBOT_INIT_JOINT_ANGLES)
while True:
    status = robot.move_towards_pose(bottleneck_pose, 0.10, 0.05)
    if status:
       break
#rospy.sleep(0.5)
#robot.move_to_pose(config.ROBOT_INIT_POSE)
#robot.move_to_joint_angles(config.ROBOT_INIT_JOINT_ANGLES)



In [ ]:
#robot.robot.set_command_timeout(1.5)
robot.move_to_pose(config.ROBOT_INIT_POSE)


        
        

#### Record cartesian velocities

In [ ]:
cart_velocities = []
while True:
    lin_vel, ang_vel = robot.get_endpoint_velocity_in_base_frame()
    cart_velocities.append(np.array([*lin_vel,*ang_vel]))
    rospy.sleep(0.5)

In [ ]:
len(cart_velocities)

In [ ]:
robot.robot.set_command_timeout(1.0)
for cart_vel in cart_velocities:
    robot.set_endpoint_velocity_in_base_frame(-cart_vel)
    rospy.sleep(0.5)
robot.robot.exit_control_mode() 

#### Record joint velocities

In [ ]:
joint_velocities = []
while True:
    joint_velocity = robot.robot.velocities()
    joint_velocities.append(joint_velocity)
    rospy.sleep(0.5)

In [ ]:
len(joint_velocities)

In [ ]:
robot.robot.set_command_timeout(1.0)
for joint_velocity in joint_velocities:
    robot.robot.exec_velocity_cmd(joint_velocity)
    rospy.sleep(0.5)
#robot.robot.exit_control_mode() 

##### Stop velocity control in between with some velocity value

In [ ]:
robot.robot.set_command_timeout(1.0)
for i, joint_velocity in enumerate(joint_velocities):
    robot.robot.exec_velocity_cmd(joint_velocity)
    rospy.sleep(0.5)
    if i == 10: exit()
robot.robot.exit_control_mode() 

#### Record Efforts/torques and replay

In [ ]:
joint_efforts = []
while True:
    joint_effort = robot.robot.efforts()
    joint_efforts.append(joint_effort)
    rospy.sleep(0.5)

In [ ]:
len(joint_efforts)

In [ ]:
robot.robot.set_command_timeout(1.0)
for joint_effort in joint_efforts:
    robot.robot.exec_torque_cmd(joint_effort)
    rospy.sleep(0.5)
#robot.robot.exit_control_mode() 